# A minimal version of N3FIT-DIS

The following notebook minimally implements what **n3fit** does (mainly DIS). For the sake of simplicity, we use as inputs saved files (`toyexpinfo.pkl` for experimental datasets, `posdatasets.pkl` for positivity datasets, and `integdatasets.pkl` for integrability datasets) generated from a **n3fit** run using the `toy-runcard.yaml`. In the current NNPDF machinery, the loading and parsing of the inputs are handled by **validphys**. Here, the only part that we care about is to illustrate how theoretical predictions are produced upon fitting the PDFs with Neural Networks and subsequently how are they compared to the experimental measurements. 

This notebook therefore tries to accomplishes two purposes: (1) provide an easy introduction to the fitting machinery of the NNPDF code by simplifying the code, (2) serve as a baseline for experimenting new and particularly subtle features. In these regards, the classes, methods, and attributes defined below are mirrored from those appearing in **n3fit** in order to trivially make a one to one compraison. Three disclaimers: 
* **n3fit** is much more modular and makes various calls from **validphys** (which makes the structure of codes complicated) and hence things might be slightly represented differently there.
*  Some of the advanced features such as **k-folding**, **hyperparameter optimization**, and **feature scaling** are not touched here.
* To facilitate various operations we are going to use the the **n3fit** backends instead of the keras backends.The **n3fit** backends provide extra-flexibility that comes very handy, one example is the ability to fix the input layer.

## 1. Definitions and notations

For **Deep Inelastic Scattering (DIS)**, observables are represented as a matrix multiplication between an FK-table
and PDFs as follows:
$$ \mathcal{O}^{\rm th}_d = \sum^{N_x}_{\alpha} \sum^{N_f}_{i} \mathrm{FK}_{\alpha i}^{(d)} \mathrm{NN}_i (x_\alpha, Q_0), $$
where $\alpha$ and $i$ denote the data point and the $x$ index respectively. $\mathrm{NN}_i (x_\alpha, Q_0)$ represents the PDF in the **evolution basis** evaluated at the initial scale $Q_0$ and at a $x$-grid point $x = x_\alpha$. $\mathrm{FK}_{\alpha i}^{(d)}$ denotes the FK table corresponding to $x = x_\alpha$ in $x$-grid for a data point $d$ and associated with the PDF $i$. Here, it is important to emphasize that every dataset contains different number of $x$ and data points. For a detaled information on how FK tables are formated in the actual NNPDF fit, refer to this [documentation](https://docs.nnpdf.science/data/th-data-files.html).

| Notation      | Default | Description                |
|---------------|---------|----------------------------|
| $N_f$         | 14      | PDF basis                  |
| $N_x$         | None    | Size of $x$-grid           |
| $N_{\rm dpt}$ | None    | Total number of datapoints |

Following the **n3fit** and **validphys** implementations, the inputs to the fit have the following shapes:

| Data                       | Description    | shape                         |
|----------------------------|----------------|-------------------------------|
| $\mathrm{FK}$              | rank-3 tensor  | ($N_{\rm dpt}$, $N_f$, $N_x$) |
| $\mathcal{O}$$^{\rm exp}$  | 1-d array      | ($N_{\rm dpt}$)               |
| $\mathrm{cov}$             | Square matrix  | ($N_{\rm dpt}$, $N_{\rm dpt}$)|

## 2. Import modules

In [1]:
import random
import pickle
import numpy as np
import tensorflow as tf
import logging

from abc import abstractmethod, ABC
from dataclasses import dataclass
from rich.console import Console
from rich.table import Table

# Use n3fit backends which are wrappers around
# tf.keras backends.
from n3fit.backends import Input
from n3fit.backends import base_layer_selector
from n3fit.backends import MetaModel
from n3fit.backends import callbacks
from n3fit.backends import MetaLayer
from n3fit.backends import operations as op
from n3fit.backends import clear_backend_state
from n3fit.stopping import Stopping

# Define seeds
random.seed(123)
np.random.seed(456)
console = Console()

log = logging.getLogger(__name__)

Using Keras backend


## 3. Load saved toy-datasets from N3FIT

In [2]:
exp_pkl_file = open("toyexpinfo.pkl", "rb")
toyexpinfo = pickle.load(exp_pkl_file)
console.print(f"Nb. Datasets: {len(toyexpinfo[0]['datasets'])}", style="bold blue")

Nb. Datasets: 2

In [3]:
pos_pkl_file = open("posdatasets.pkl", "rb")
toyposdatasets = pickle.load(pos_pkl_file)

In [4]:
integ_pkl_file = open("integdatasets.pkl", "rb")
toyintegdatasets = pickle.load(integ_pkl_file)

## 4. Create the NN architectures

### 4.1 Construct the hidden layers

In [5]:
def generate_dense_network(
    nodes_in,
    nodes,
    activations,
    initializer_name="glorot_normal",
    seed=0,
    dropout_rate=0.0,
    regularizer=None,
):
    """This function generate the different `tf.keras.layers.Dense` layers and add
    them to a list. The number of layers depends on the length of the `node_per_layer`.
    This function mimicks the behaviour of the `generate_dense_network` function in 
    the `n3fit.model_gen` module.
    
    Returns:
    --------
    list:
        List of `tf.keras.layers.Dense` layers.
    """
    
    choose_keras_layer_type = "dense"
    list_of_layers = []
    for (nodes_out, activation) in zip(nodes, activations):
        list_of_layers.append(base_layer_selector(
            choose_keras_layer_type,
            units=int(nodes_out),
            input_shape=(nodes_in,),
            kernel_initializer=initializer_name,
            activation=activation
        ))
        # Make sure that the next layer has the same input nodes as the
        # output of the previous layer
        nodes_in = int(nodes_out)
    return list_of_layers

### 4.2 Construct the complete NN architecture

In [6]:
from n3fit.layers import DIS, DY, ObsRotation, losses
from n3fit.layers import Preprocessing, FkRotation, FlavourToEvolution
from n3fit.backends import MetaLayer, Lambda
from n3fit.backends import base_layer_selector, regularizer_selector

def pdfNN_layer_generator(
    inp=2,
    nodes=None,
    activations=None,
    initializer_name="glorot_normal",
    layer_type="dense",
    flav_info=None,
    fitbasis="NN31IC",
    out=14,
    seed=None,
    dropout=0.0,
    regularizer=None,
    regularizer_args=None,
    impose_sumrule=None,
    scaler=None,
    parallel_models=1,
):
    """This function constructs the main Neural Network to fit the PDFs. This part
    mimicks the function `pdfNN_layer_generator` present in `n3fit.model_gen` module
    by simplifying it greatly, i.e. removing all the details such as Evolution,
    basis rotation, and sum rules. Notice that these layers are not trained during
    the fitting procedure and therefore are immaterial here.
    """

    number_of_layers = len(nodes)
    # Last layer fixed to be fl=14 (flavour basis)
    last_layer_nodes = nodes[-1]

    # First prepare the input for the PDF model (no scaling)
    placeholder_input = Input(shape=(None, 1), batch_size=1)
    
    process_input = Lambda(lambda x: op.concatenate([x, op.op_log(x)], axis=-1))

    model_input = [placeholder_input]

    layer_evln = lambda x: x       # Mimick Evolution
    basis_rotation = lambda x: x   # Mimick Basis Rotation
    sumrule_layer = lambda x: x    # Mimick Sum Rules

    pdf_models = []
    for i, layer_seed in enumerate(seed):
        list_of_pdf_layers = generate_dense_network(
            inp,
            nodes,
            activations,
            initializer_name,
            seed=layer_seed,
            dropout_rate=dropout,
            regularizer=None,
        )

        def dense_me(x):
            """Function that takes a list of layers and connect them."""
            processed_x = process_input(x)
            curr_fun = list_of_pdf_layers[0](processed_x)

            for dense_layer in list_of_pdf_layers[1:]:
                curr_fun = dense_layer(curr_fun)
            return curr_fun

        def layer_fitbasis(x):
            """Funccation that performs the roation to change the basis."""
            x_scaled = op.op_gather_keep_dims(x, 0, axis=-1)
            x_original = op.op_gather_keep_dims(x, -1, axis=-1)
            nn_output = dense_me(x_scaled)
            return basis_rotation(nn_output)

        def layer_pdf(x):
            return layer_evln(layer_fitbasis(x))

        # Final PDF (apply normalization)
        final_pdf = sumrule_layer(layer_pdf)

        # Create the model
        pdf_model = MetaModel(model_input, final_pdf(placeholder_input), name=f"PDF_{i}", scaler=scaler)
        pdf_models.append(pdf_model)
    return pdf_models

### 4.3 Construct the Observable

Now, we can implement the part that computes the Observable ($\mathcal{O}^{\rm th}$) expressed in the equation above. Due to the way how the observables are computed, the standard *tf.keras.layers* will no longer work, instead, one has to define a custom layer which inherits from the *tf.keras.layers.Layer*.

In [7]:
def _is_unique(list_of_arrays):
    """This function takes a list of numpy arrays and checks whether it contains
    more tha one different arrays.
    """
    the_first = list_of_arrays[0]
    for i in list_of_arrays[1:]:
        if not np.array_equal(the_first, i):
            return False
    return True


class Observable(MetaLayer, ABC):
    """Abstract classs that initializes the relevant inputs for the computation
    of the theoretical predictions. This Class does not yet have a method that
    actually computes the Observable.
    
    This class is an almost exact copy of the class `Observable` contained in the
    module `n3fit.layers.observable`.
    
    Inputs:
    ------
    list_fktable_dicts: list
        list of dictionaries where each element contains the specifications about
        a given FK table. Each dictionary has the following keys:
        - ndata: nb of datapoints
        - nbasis:
        - nonzero: nb o non-zero PDF entries
        - basis: gives the index of the non-zero PDFs
        - nx: size of the x-grid
        - xgrid: array of x-points
        - fktable: full/unmasked FK table of shape (ndata, nonzero, nx)
            
    list_fktable_mask: list of boolean arrays
        list of training/validation mask to be applied to the fktable/observable
        
    operation_name: str
        operation to be performed for a given FK table(s)
    """

    def __init__(self, fktable_dicts, operation_name, nfl=14, **kwargs):
        
        super(MetaLayer, self).__init__(**kwargs)

        self.nfl = nfl
        basis, xgrids, self.fktables = [], [], []

        for fktable in fktable_dicts:
            xgrids.append(fktable["xgrid"])
            basis.append(fktable["basis"])
            self.fktables.append(fktable["fktable"])
            
        # check how many xgrids this dataset needs
        if _is_unique(xgrids): self.splitting = None
        else: self.splitting = [i.shape[1] for i in xgrids]

        # check how many basis this dataset needs
        if _is_unique(basis) and _is_unique(xgrids):
            self.all_masks = [self.gen_mask(basis[0])]
            self.many_masks = False
        else:
            self.many_masks = True
            self.all_masks = [self.gen_mask(i) for i in basis]

        self.operation = op.c_to_py_fun(operation_name)
        self.output_dim = self.fktables[0].shape[0]

    def compute_output_shape(self, input_shape):
        return (self.output_dim, None)

    @abstractmethod
    def gen_mask(self, basis):
        """This method needs to be initialized here in order for it to get
        overwritten when it is called elsewhere.
        """
        pass

In [8]:
class DIS(Observable):
    """Class that inherits from the Observable Layer class. It fetches all the relevant
    variables from the parent class, overwrites the masks, and compute the theory predictions
    using the formula written above. This class is an almost exact copy of the `DIS` class
    in the `n3fit.layers` module.
    """

    def gen_mask(self, basis):
        """Takes a list of the index of the non-zero PDFs and return a boolean mask."""
        
        if basis is None:
            self.basis = np.ones(self.nfl, dtype=bool)
        else:
            basis_mask = np.zeros(self.nfl, dtype=bool)
            for i in basis:
                basis_mask[i] = True
        return op.numpy_to_tensor(basis_mask, dtype=bool)
    

    def call(self, pdf):
        """Performs the convolution between an FK table with a PDF.
        
        Ingredients:
        ------------
        self.fktables: list
            list of masked FK tables (for training) where each FK tables has a shape 
            (ndata_masked, non_zero_fl, nx)
            
        pdf: list
            list of PDF tensor (output of the NN) with shape (batch_size, nx, non_zero_fl)
        """
        
        if self.splitting is not None:
            raise ValueError("Splitting should always be None for DIS.")

        results = []
        if self.many_masks:
            for mask, fktable in zip(self.all_masks, self.fktables):
                # We also need to mask the PDFs output from the NN. Recall that 
                # the output of the NN is the 14 PDFs in the flavour basis
                pdf_masked = op.boolean_mask(pdf, mask, axis=2)
                res = op.tensor_product(pdf_masked, fktable, axes=[(1, 2), (2, 1)])
                results.append(res)
        else:
            pdf_masked = op.boolean_mask(pdf, self.all_masks[0], axis=2)
            for fktable in self.fktables:
                res = op.tensor_product(pdf_masked, fktable, axes=[(1, 2), (2, 1)])
                results.append(res)
        
        res = self.operation(results)
        return res

As in the case of the PDFs `pdfNN_layer_generator`, we need an observable generator. The way **n3fit** this is to first define an `Observable` wrapper that constructs the graph for the computation of the **loss functions**. Here, since we are not yet interested in the form of the loss functions, we are just going to rely on the `losses` module of **n3fit**. In particular, we are only going to use the `LossInvcovmat` class.

In [9]:
from n3fit.layers.losses import LossPositivity
from n3fit.layers.losses import LossInvcovmat
from n3fit.layers.losses import LossIntegrability

In [10]:
@dataclass
class ObservableWrapper:
    """This class mimicks the `ObservableWrapper` class in the module `n3fit.model_gen`."""
    
    name: str
    observables: list
    masks: list
    dataset_xsizes: list
    invcovmat: np.array = None
    covmat: np.array = None
    multiplier: float = 1.0
    integrability: bool = False
    positivity: bool = False
    data: np.array = None
    rotation: ObsRotation = None

    def _generate_loss(self, mask=None):
        """Returns `n3fit.backends.layer.MetaLayer` of the loss function."""
        
        if self.invcovmat is not None:
            for d, m in zip(self.data, self.masks):
                console.log("The incovmat input data is now", d.shape)
            if m is not None:
                console.log("The input trvl mask is now", m.mask.shape)
            #mask = self.masks[0].mask if self.masks[0] is not None else None
            loss = losses.LossInvcovmat(self.invcovmat, self.data, mask, name=self.name)
        elif self.positivity:
            loss = losses.LossPositivity(name=self.name, c=self.multiplier)
        elif self.integrability:
            loss = losses.LossIntegrability(name=self.name, c=self.multiplier)
        return loss

    
    def _generate_experimental_layer(self, pdf):
        """Generates the experimental layer from the PDF"""

        if len(self.dataset_xsizes) > 1:
            splitting_layer = op.as_layer(
                op.split,
                op_args=[self.dataset_xsizes],
                op_kwargs={"axis": 1},
                name=f"{self.name}_split",
            )
            split_pdf = splitting_layer(pdf)
        else: split_pdf = [pdf]
        
        output_layers = []
        for p_pdf, obs, mask in zip(split_pdf, self.observables, self.masks):
            if mask is not None:
                output_layers.append(mask(obs(p_pdf)))
            else:
                output_layers.append(obs(p_pdf))
        ret = op.concatenate(output_layers, axis=2)
        if self.rotation is not None: ret = self.rotation(ret)
        return ret

    def __call__(self, pdf_layer, mask=None):
        loss_f = self._generate_loss(mask)
        experiment_prediction = self._generate_experimental_layer(pdf_layer)
        return loss_f(experiment_prediction)

In [11]:
from n3fit.layers import Mask

def observable_generator(
    spec_dict, positivity_initial=1.0, integrability=False
):
    
    spec_name = spec_dict["name"]
    dataset_xsizes = []
    model_inputs = []
    model_obs_layers = []
    tr_mask_layers = []
    vl_mask_layers = []

    offset = 0

    obs_layer_tr, obs_layer_vl, obs_layer_ex = None, None, None
    mask_tr, mask_vl  = None, None
    
    apply_masks = spec_dict.get("data_transformation_tr") is None
    
    # The first step is to compute the observable for each of the datasets
    for dataset_dict in spec_dict["datasets"]:
        # Get the generic information of the dataset
        dataset_name = dataset_dict["name"]
        trmask = spec_dict["trmask"][offset:offset + dataset_dict["ndata"]]
        tr_mask_layers.append(Mask(trmask, axis=2) if apply_masks else None)
        vl_mask_layers.append(Mask(~trmask, axis=2) if apply_masks else None)

        # Instantiate the DIS Observable 
        obs_layer = DIS(dataset_dict["fktables"],
                        dataset_dict["operation"],
                        name=f"dat_{dataset_name}")
        model_obs_layers.append(obs_layer)

        if obs_layer.splitting is None:
            xgrid = dataset_dict["fktables"][0]["xgrid"]
            model_inputs.append(xgrid)
            dataset_xsizes.append(xgrid.shape[1])
        else:
            xgrids = [i["xgrid"] for i in dataset_dict["fktables"]]
            model_inputs += xgrids
            dataset_xsizes.append(sum([i.shape[1] for i in xgrids]))
        
        # shift offset for new mask array
        offset = offset + dataset_dict["ndata"]

    full_nx = sum(dataset_xsizes)
    
    if spec_dict["positivity"]:
        out_positivity = ObservableWrapper(
            spec_name,
            model_obs_layers,
            tr_mask_layers,
            dataset_xsizes,
            multiplier=positivity_initial,
            positivity=not integrability,
            integrability=integrability,
        )

        layer_info = {
            "inputs": model_inputs,
            "output_tr": out_positivity,
            "experiment_xsize": full_nx,
        }
        # For positivity we end here
        return layer_info

    if spec_dict.get("data_transformation_tr") is not None:
        obsrot_tr = ObsRotation(spec_dict.get("data_transformation_tr"))
        obsrot_vl = ObsRotation(spec_dict.get("data_transformation_vl"))
    else:
        obsrot_tr = None
        obsrot_vl = None

    #console.log("expdata:",spec_dict["expdata"].shape)
    #console.log("expdata_vl:",spec_dict["expdata_vl"].shape)
    #console.log("expdata_true:",spec_dict["expdata_true"].shape)
    
    out_tr = ObservableWrapper(
        spec_name,
        model_obs_layers,
        tr_mask_layers,
        dataset_xsizes,
        invcovmat=spec_dict["invcovmat"],
        data=spec_dict["expdata"],
        rotation=obsrot_tr,
    )
    out_vl = ObservableWrapper(
        f"{spec_name}_val",
        model_obs_layers,
        vl_mask_layers,
        dataset_xsizes,
        invcovmat=spec_dict["invcovmat_vl"],
        data=spec_dict["expdata_vl"],
        rotation=obsrot_vl,
    )
    out_exp = ObservableWrapper(
        f"{spec_name}_exp",
        model_obs_layers,
        [None] * len(model_obs_layers),
        dataset_xsizes,
        invcovmat=spec_dict["invcovmat_true"],
        covmat=spec_dict["covmat"],
        data=spec_dict["expdata_true"],
        rotation=None,
    )

    layer_info = {
        "inputs": model_inputs,
        "output": out_exp,
        "output_tr": out_tr,
        "output_vl": out_vl,
        "experiment_xsize": full_nx,
    }
    return layer_info

## 5. Compile the Models & Fit

What remains to do now is to combined everything and construct a class that controls and performs a fit.

In [12]:
from itertools import zip_longest

def _pdf_injection(pdf_layers, observables, masks):
    """Takes as input a list of PDF layers and if needed applies masks."""
    return [f(x, mask=m) for f, x, m in zip_longest(observables, pdf_layers, masks)]

In [13]:
PUSH_POSITIVITY_EACH = 100
PUSH_INTEGRABILITY_EACH = 100
CHI2_THRESHOLD = 10.0

def _LM_initial_and_multiplier(input_initial, input_multiplier, max_lambda, steps):
    initial, multiplier = input_initial, input_multiplier
    if multiplier is None:
        if initial is None: initial = 1.0
        multiplier = pow(max_lambda / initial, 1 / max(steps, 1))
    elif initial is None:
        initial = max_lambda / pow(multiplier, steps)
    return initial, multiplier

class ModelTrainer:

    def __init__(
        self,
        exp_info,
        pos_info,
        integ_info,
        flavinfo,
        fitbasis,
        nnseeds,
        pass_status="ok",
        failed_status="fail",
        debug=False,
        kfold_parameters=None,
        max_cores=None,
        model_file=None,
        sum_rules=None,
        parallel_models=1,
    ):
        
        self.exp_info = exp_info
        self.pos_info = pos_info
        self.integ_info = integ_info
        if self.integ_info is not None:
            self.all_info = exp_info + pos_info + integ_info
        else:
            self.all_info = exp_info + pos_info
        self.flavinfo = flavinfo
        self.fitbasis = fitbasis
        self._nn_seeds = nnseeds
        self.pass_status = pass_status
        self.failed_status = failed_status
        self.debug = debug
        self.all_datasets = []
        self._scaler = None
        self._parallel_models = parallel_models
        self.impose_sumrule = sum_rules
        self.kpartitions = [None]

        self.input_list = []
        self.input_sizes = []
        self.training = {
            "output": [],
            "expdata": [],
            "ndata": 0,
            "model": None,
            "posdatasets": [],
            "posmultipliers": [],
            "posinitials": [],
            "integdatasets": [],
            "integmultipliers": [],
            "integinitials": [],
            "folds": [],
        }
        self.validation = {
            "output": [],
            "expdata": [],
            "ndata": 0,
            "model": None,
            "folds": [],
            "posdatasets": [],
        }
        self.experimental = {
            "output": [],
            "expdata": [],
            "ndata": 0,
            "model": None,
            "folds": [],
        }

        self._fill_the_dictionaries()

        if self.validation["ndata"] == 0:
            self.no_validation = True
            self.validation["expdata"] = self.training["expdata"]
        else:
            self.no_validation = False
            
        self.callbacks = [callbacks.TimerCallback()]
            

    def _model_generation(self, pdf_models, partition, partition_idx):
       
        log.info("Generating the Model")

        # Construct the input array that will be given to the pdf
        input_arr = np.concatenate(self.input_list, axis=1).T
        input_layer = op.numpy_to_input(input_arr)

        all_replicas_pdf = []
        for pdf_model in pdf_models:
            full_model_input_dict, full_pdf = pdf_model.apply_as_layer([input_layer])
            all_replicas_pdf.append(full_pdf)

        full_pdf_per_replica = op.stack(all_replicas_pdf, axis=-1)

        splitted_pdf = op.as_layer(
            op.split, 
            op_args=[self.input_sizes], 
            op_kwargs={"axis": 1}, 
            name="pdf_split"
        )(full_pdf_per_replica)

        training_mask = validation_mask = experimental_mask = [None]

        output_tr = _pdf_injection(splitted_pdf, self.training["output"], training_mask)
        training = MetaModel(full_model_input_dict, output_tr)

        val_pdfs, exp_pdfs = [], []
        for partial_pdf, obs in zip(splitted_pdf, self.training["output"]):
            if not obs.positivity and not obs.integrability:
                val_pdfs.append(partial_pdf)
                exp_pdfs.append(partial_pdf)
            elif not obs.integrability and obs.positivity:
                val_pdfs.append(partial_pdf)

        # We don't want to included the integrablity in the validation
        output_vl = _pdf_injection(val_pdfs, self.validation["output"], validation_mask)
        validation = MetaModel(full_model_input_dict, output_vl)
        output_ex = _pdf_injection(exp_pdfs, self.experimental["output"], experimental_mask)
        experimental = MetaModel(full_model_input_dict, output_ex)

        # exp_model = MetaModel(full_model_input_dict, output_ex)
        # vl_model = exp_model.append(vl_mask_layer)
        # tr_model  = 
        
        training.summary()

        return {
            "training": training,
            "validation": validation,
            "experimental": experimental,
        }


    def _generate_observables(
        self,
        all_pos_multiplier,
        all_pos_initial,
        all_integ_multiplier,
        all_integ_initial,
        epochs,
        interpolation_points,
    ):
        """
        This functions fills the 3 dictionaries (training, validation, experimental)
        with the output layers and the loss functions
        It also fill the list of input tensors (input_list)

        The arguments of this function are used to define the initial positivity of the
        positivity observables and the multiplier to be applied at each step.

        Parameters
        ----------
            all_pos_multiplier: float, None
                multiplier to be applied to the positivity each ``PUSH_POSITIVITY_EACH`` epochs
            all_pos_initial: float, None
                initial value for the positivity lambda
            epochs: int
                total number of epochs for the run
        """

        # First reset the dictionaries
        self._reset_observables()
        console.print("Generating layers", style="bold red")
        
        for exp_dict in self.exp_info:
            exp_layer = observable_generator(exp_dict)

            self.input_list += exp_layer["inputs"]
            self.input_sizes.append(exp_layer["experiment_xsize"])

            self.training["output"].append(exp_layer["output_tr"])
            self.validation["output"].append(exp_layer["output_vl"])
            self.experimental["output"].append(exp_layer["output"])

        for pos_dict in self.pos_info:
            positivity_steps = int(epochs / PUSH_POSITIVITY_EACH)
            max_lambda = pos_dict["lambda"]

            pos_initial, pos_multiplier = _LM_initial_and_multiplier(
                all_pos_initial, 
                all_pos_multiplier, 
                max_lambda, 
                positivity_steps
            )

            pos_layer = observable_generator(pos_dict, positivity_initial=pos_initial)
            self.input_list += pos_layer["inputs"]
            self.input_sizes.append(pos_layer["experiment_xsize"])

            self.training["output"].append(pos_layer["output_tr"])
            self.validation["output"].append(pos_layer["output_tr"])
            self.training["posmultipliers"].append(pos_multiplier)
            self.training["posinitials"].append(pos_initial)

        if self.integ_info is not None:
            for integ_dict in self.integ_info:
                integrability_steps = int(epochs / PUSH_INTEGRABILITY_EACH)
                max_lambda = integ_dict["lambda"]

                integ_initial, integ_multiplier = _LM_initial_and_multiplier(
                    all_integ_initial, 
                    all_integ_multiplier, 
                    max_lambda, 
                    integrability_steps
                )

                integ_layer = observable_generator(
                    integ_dict, 
                    positivity_initial=integ_initial, 
                    integrability=True
                )
                self.input_list += integ_layer["inputs"]
                self.input_sizes.append(integ_layer["experiment_xsize"])
                self.training["output"].append(integ_layer["output_tr"])
                self.training["integmultipliers"].append(integ_multiplier)
                self.training["integinitials"].append(integ_initial)


    def _generate_pdf(
        self,
        nodes_per_layer,
        activation_per_layer,
        initializer,
        layer_type,
        dropout,
        regularizer,
        regularizer_args,
        seed,
    ):
        """Generate the PDF NNs"""
        
        console.print("Generate PDF models", style="bold red")

        pdf_models = pdfNN_layer_generator(
            nodes=nodes_per_layer,
            activations=activation_per_layer,
            layer_type=layer_type,
            flav_info=self.flavinfo,
            fitbasis=self.fitbasis,
            seed=seed,
            initializer_name=initializer
        )
        return pdf_models
    

    def _train_and_fit(self, training_model, stopping_object, epochs=100):
        """
        Trains the NN for the number of epochs given using
        stopping_object as the stopping criteria

        Every ``PUSH_POSITIVITY_EACH`` epochs the positivity will be multiplied by their
        respective positivity multipliers.
        In the same way, every ``PUSH_INTEGRABILITY_EACH`` epochs the integrability
        will be multiplied by their respective integrability multipliers
        """
        callback_st = callbacks.StoppingCallback(stopping_object)
        callback_pos = callbacks.LagrangeCallback(
            self.training["posdatasets"],
            self.training["posmultipliers"],
            update_freq=PUSH_POSITIVITY_EACH,
        )
        callback_integ = callbacks.LagrangeCallback(
            self.training["integdatasets"],
            self.training["integmultipliers"],
            update_freq=PUSH_INTEGRABILITY_EACH,
        )

        training_model.perform_fit(
            epochs=epochs,
            verbose=False,
            callbacks=self.callbacks + [callback_st, callback_pos, callback_integ],
        )

        if any(bool(i) for i in stopping_object.e_best_chi2):
            return self.pass_status
        return self.failed_status

    def evaluate(self, stopping_object):
        """Returns the training, validation and experimental chi2

        Parameters
        ----------
            stopping_object
                A Stopping intance which will have associated a validation model and the
                list of output layers that should contribute to the training chi2

        Returns
        -------
            train_chi2: chi2 of the trainining set
            val_chi2 : chi2 of the validation set
            exp_chi2: chi2 of the experimental data (without replica or tr/vl split)
        """
        if self.training["model"] is None:
            raise RuntimeError("Modeltrainer.evaluate was called before any training")
        train_chi2 = stopping_object.evaluate_training(self.training["model"])
        val_chi2 = stopping_object.vl_chi2
        exp_chi2 = self.experimental["model"].compute_losses()["loss"] / self.experimental["ndata"]
        return train_chi2, val_chi2, exp_chi2

    def hyperparametrizable(self, params):
        """
        Wrapper around all the functions defining the fit.

        After the ModelTrainer class has been instantiated,
        a call to this function (with a ``params`` dictionary) is necessary
        in order to generate the whole PDF model and perform a fit.

        This is a necessary step for hyperopt to work

        Parameters used only here:
            - ``epochs``: maximum number of iterations for the fit to run
            - ``stopping_patience``: patience of the stopper after finding a new minimum
        All other parameters are passed to the corresponding functions
        """

        clear_backend_state()

        epochs = int(params["epochs"])
        stopping_patience = params["stopping_patience"]
        stopping_epochs = int(epochs * stopping_patience)

        positivity_dict = params.get("positivity", {})
        integrability_dict = params.get("integrability", {})
        self._generate_observables(
            positivity_dict.get("multiplier"),
            positivity_dict.get("initial"),
            integrability_dict.get("multiplier"),
            integrability_dict.get("initial"),
            epochs,
            params.get("interpolation_points"),
        )
        threshold_pos = positivity_dict.get("threshold", 1e-6)
        threshold_chi2 = params.get("threshold_chi2", CHI2_THRESHOLD)

        ### Training loop
        for k, partition in enumerate(self.kpartitions):
            seeds = self._nn_seeds

            pdf_models = self._generate_pdf(
                params["nodes_per_layer"],
                params["activation_per_layer"],
                params["initializer"],
                params["layer_type"],
                params["dropout"],
                params.get("regularizer", None),
                params.get("regularizer_args", None),
                seeds,
            )

            models = self._model_generation(pdf_models, partition, k)

            reporting = self._prepare_reporting(partition)

            if self.no_validation:
                models["validation"] = models["training"]
                validation_model = models["training"]
            else:
                validation_model = models["validation"]

            stopping_object = Stopping(
                validation_model,
                reporting,
                pdf_models,
                total_epochs=epochs,
                stopping_patience=stopping_epochs,
                threshold_positivity=threshold_pos,
                threshold_chi2=threshold_chi2,
            )

            # Compile each of the models with the right parameters
            for model in models.values():
                model.compile(**params["optimizer"])

            passed = self._train_and_fit(
                models["training"],
                stopping_object,
                epochs=epochs,
            )

        self.training["model"] = models["training"]
        self.experimental["model"] = models["experimental"]
        self.validation["model"] = models["validation"]

        dict_out = {"status": passed, "stopping_object": stopping_object, "pdf_models": pdf_models}
        return dict_out
    
    #--------------------------------------------------#
    #                   VARIOUS UTILS                  #
    #--------------------------------------------------#
    
    def _reset_observables(self):
        """Reset the various lists."""
        
        self.input_list = []
        self.input_sizes = []
        for key in ["output", "posmultipliers", "integmultipliers"]:
            self.training[key] = []
            self.validation[key] = []
            self.experimental[key] = []
            
    
    def _fill_the_dictionaries(self):
        """Transfer information from `self.exp_info` into the dictionaries."""
        
        for exp_dict in self.exp_info:
            self.training["expdata"].append(exp_dict["expdata"])
            self.validation["expdata"].append(exp_dict["expdata_vl"])
            self.experimental["expdata"].append(exp_dict["expdata_true"])

            self.training["folds"].append(exp_dict["folds"]["training"])
            self.validation["folds"].append(exp_dict["folds"]["validation"])
            self.experimental["folds"].append(exp_dict["folds"]["experimental"])

            nd_tr = exp_dict["ndata"]
            nd_vl = exp_dict["ndata_vl"]

            self.training["ndata"] += nd_tr
            self.validation["ndata"] += nd_vl
            self.experimental["ndata"] += nd_tr + nd_vl

            for dataset in exp_dict["datasets"]:
                self.all_datasets.append(dataset["name"])
        self.all_datasets = set(self.all_datasets)

        for pos_dict in self.pos_info:
            self.training["expdata"].append(pos_dict["expdata"])
            self.training["posdatasets"].append(pos_dict["name"])
            self.validation["expdata"].append(pos_dict["expdata"])
            self.validation["posdatasets"].append(pos_dict["name"])
        if self.integ_info is not None:
            for integ_dict in self.integ_info:
                self.training["expdata"].append(integ_dict["expdata"])
                self.training["integdatasets"].append(integ_dict["name"])
                
    def _prepare_reporting(self, partition):
        """Parses the information received by the :py:class:`n3fit.ModelTrainer.ModelTrainer`
        to select the bits necessary for reporting the chi2.
        Receives the chi2 partition data to see whether any dataset is to be left out
        """
        reported_keys = ["name", "count_chi2", "positivity", "integrability", "ndata", "ndata_vl"]
        reporting_list = []
        for exp_dict in self.all_info:
            reporting_dict = {k: exp_dict.get(k) for k in reported_keys}
            reporting_list.append(reporting_dict)
        return reporting_list

## 6. Peform Fit

In [16]:
nnseed = [1872583848] * 5
fitbasis = 'EVOL'
model_file = None

flav_info = [
    {'fl': 'sng', 'trainable': False, 'smallx': [1.094, 1.118], 'largex': [1.46, 3.003]}, 
    {'fl': 'g', 'trainable': False, 'smallx': [0.8189, 1.044], 'largex': [2.791, 5.697]}, 
    {'fl': 'v', 'trainable': False, 'smallx': [0.457, 0.7326], 'largex': [1.56, 3.431]}, 
    {'fl': 'v3', 'trainable': False, 'smallx': [0.1462, 0.4061], 'largex': [1.745, 3.452]}, 
    {'fl': 'v8', 'trainable': False, 'smallx': [0.5401, 0.7665], 'largex': [1.539, 3.393]}, 
    {'fl': 't3', 'trainable': False, 'smallx': [-0.4401, 0.9163], 'largex': [1.773, 3.333]}, 
    {'fl': 't8', 'trainable': False, 'smallx': [0.5852, 0.8537], 'largex': [1.533, 3.436]}, 
    {'fl': 't15', 'trainable': False, 'smallx': [1.082, 1.142], 'largex': [1.461, 3.1]}
]

params = {
    'nodes_per_layer': [15, 10, 14], 
    'activation_per_layer': ['sigmoid', 'sigmoid', 'linear'], 
    'initializer': 'glorot_normal', 
    'optimizer': {'optimizer_name': 'RMSprop', 'learning_rate': 0.01, 'clipnorm': 1.0}, 
    'epochs': 900, 'positivity': {'multiplier': 1.05, 'initial': None, 'threshold': 1e-05}, 
    'stopping_patience': 0.3, 'layer_type': 'dense', 'dropout': 0.0, 'threshold_chi2': 5.0
}

In [17]:
ModelTraining = ModelTrainer(
    toyexpinfo,
    toyposdatasets,
    toyintegdatasets,
    flav_info,
    fitbasis,
    nnseed
)

results = ModelTraining.hyperparametrizable(params)

Generating layers

Generate PDF models

[14:01:46] The incovmat input data is now                                                          ]8;id=767598;file:///tmp/ipykernel_22307/1194756364.py\1194756364.py]8;;\:]8;id=470099;file:///tmp/ipykernel_22307/1194756364.py#22\22]8;;\
           (76,)                                                                                                   

           The input trvl mask is now (121,)                                                       ]8;id=699545;file:///tmp/ipykernel_22307/1194756364.py\1194756364.py]8;;\:]8;id=149081;file:///tmp/ipykernel_22307/1194756364.py#24\24]8;;\

[14:01:47] The incovmat input data is now                                                          ]8;id=305976;file:///tmp/ipykernel_22307/1194756364.py\1194756364.py]8;;\:]8;id=955410;file:///tmp/ipykernel_22307/1194756364.py#22\22]8;;\
           (78,)                                                                                                   

           The input trvl mask is now (121,)                                                       ]8;id=492089;file:///tmp/ipykernel_22307/1194756364.py\1194756364.py]8;;\:]8;id=878536;file:///tmp/ipykernel_22307/1194756364.py#24\24]8;;\

           The incovmat input data is now                                                          ]8;id=547010;file:///tmp/ipykernel_22307/1194756364.py\1194756364.py]8;;\:]8;id=840430;file:///tmp/ipykernel_22307/1194756364.py#22\22]8;;\
           (154,)                                                                                                  

Model: "meta_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(1, 290, 1)]        0                                            
__________________________________________________________________________________________________
PDF_0 (MetaModel)               (1, None, 14)        359         input_2[0][0]                    
__________________________________________________________________________________________________
PDF_1 (MetaModel)               (1, None, 14)        359         input_2[0][0]                    
__________________________________________________________________________________________________
PDF_2 (MetaModel)               (1, None, 14)        359         input_2[0][0]                    
_________________________________________________________________________________________

## 7. Tasks

### 7.1 Flavour Map & Convolution

Recall that the convolution (matri multplication) of the the PDFs with the FK tables are computed as follows:
$$ \mathcal{O}^{\rm th}_d = \sum^{N_x}_{\alpha} \sum^{N_f}_{i} \mathrm{FK}_{\alpha i}^{(d)} \mathrm{NN}_i (x_\alpha, Q_0), $$
Since for a given dataset/process, not all the flavours are active, therefore some of the flavours are pre-set to zero in the FK tables and are used to skip some parts of the computation.

Understand how this flavour map works and how this gets propagated in the code. You might want to look more closely at the `fktable_dicts` entry in the class `Observable`. For a very brief detailed on how FK tables are structured, have a look at this [section](https://docs.nnpdf.science/data/th-data-files.html?highlight=flavourmap#fk-table-compression).

### 7.2 Towards fitting structure functions

In order to fit structure functions, the observable that we are interested in is generally the double differential cross section expressed as:

$$ \frac{d^2 \sigma^2_{\nu N}}{dx dQ^2} = f(Q^2, m_W^2) [ Y_+ F_2^{\nu N} (x,Q^2) + Y_- F_3^{\nu N} (x,Q^2) - y^2 F_L^{\nu N} (x,Q^2) ]. $$

How would the FK tables and the flavour map change in this scenario? Think of ways in which one can adapt the FK tables and flavour map concepts to take the above into account.